In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19  # Change the import statement
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import tensorflow as tf
from keras.models import load_model
from glob import glob
import matplotlib.pyplot as plt
import os
import cv2


In [2]:

# Re-size all the images to the given size
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Sign Language for Numbers/train'
test_path = '/content/drive/MyDrive/Sign Language for Numbers/test'


In [3]:

# Add preprocessing layer to the front of VGG19
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)  # Use VGG19

# For not training existing weights
for layer in vgg.layers:
    layer.trainable = False

80134624/80134624 [==============================] - 3s 0us/step


In [4]:

# Useful for getting the number of classes
folders = glob('/content/drive/MyDrive/Sign Language for Numbers/train/*')
len(folders)

10

In [5]:

# Layers - you can add more if needed
x = Flatten()(vgg.output)
x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [6]:

# Create a model object
model = Model(inputs=vgg.input, outputs=prediction)


In [7]:


# View the structure of the model
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:

from keras import optimizers
# Tell the model what cost and optimization method to use
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])


In [9]:

train_datagen = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Sign Language for Numbers/train',
                                                 target_size=(224, 224),
                                                 batch_size=16,
                                                 class_mode='categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Sign Language for Numbers/test',
                                            target_size=(224, 224),
                                            batch_size=16,
                                            class_mode='categorical')

Found 11998 images belonging to 10 classes.
Found 3000 images belonging to 10 classes.


In [10]:

cla = test_set.classes
print(cla)

[0 0 0 ... 9 9 9]


In [11]:


r = model.fit(training_set, validation_data=test_set,
              epochs=1,
              steps_per_epoch=len(training_set),
              validation_steps=len(test_set))


KeyboardInterrupt: 

In [ ]:

# Calculate the loss in a graph
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='test loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss.jpg')

In [ ]:

# Accuracies curve in graphs
plt.plot(r.history['accuracy'], label='Train Accuracy')
plt.plot(r.history['val_accuracy'], label='Test Accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:

import matplotlib.pyplot as plt
plt.plot(r.history["accuracy"])
plt.plot(r.history['val_accuracy'])
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy", "loss", "Validation Loss"])
plt.show()

In [ ]:



model.save('/content/drive/MyDrive/Sign Language for Numbers/SignCount_VGG19_Model.h5')
print("Your model is in ==> ", os.getcwd())


In [ ]:


import cv2

img = cv2.imread("/content/drive/MyDrive/Sign Language for Numbers/train/2/two_1004.jpg")
img = img/255
plt.imshow(img)


In [ ]:

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/Sign Language for Numbers/train',
    validation_split=0.17, seed=123,
    subset="training"
)

In [ ]:


path = "/content/drive/MyDrive/Sign Language for Numbers/test/4/four_1206.jpg"

In [ ]:



from keras.preprocessing import image
from keras.utils import load_img

img = load_img(path, target_size=(224, 224))
img = np.asarray(img)
plt.imshow(img)

In [ ]:


img = np.expand_dims(img, axis=0)
output = model.predict(img)
cls = train_ds.class_names
output = cls[np.argmax(output)]
print(output)

In [ ]:

output = np.argmax(output)
print(output)

In [ ]:




cls = train_ds.class_names
cls[np.argmax(output)]
